# Задача **"Классификация тональности текста"**


---

**Выполнил**: Егор Юмаев

## **1. Введение**

---

Неотъемлемой частью современного мира являются электронные площадки, сервисы, порталы, на которых пользователи могут оставлять свои отзывы. Тональность отзывов может варьироваться от доброжелательной до негативной.

Ручной анализ комментариев пользователей может быть эффективен только при небольших количествах письменных откликов. При наличии большого потока сообщений необходимо внедрение автоматизированной системы определения тональности комментариев.

Задача оценки тональности текста является задачей **классификации**, в которой положительному тексту может быть присвоено значение "1", негативному - "0". Признаки представлены словами из корпуса и их величинами TF-IDF для каждого конкретного текста сообщения.

## **2. Задание**


---

Требуется обучить логистическую регрессию для определения тональности текста.

Исходные данные представлены двумя датасетами (обучающим и тестовым), включающими тексты комментариев пользователей в исходном и лемматизированном виде. Целевой бинарный признак 'positive' содержится в обучающем датасете.

Предсказания необходимо сохранить в столбце positive и преобразовать в отдельный csv-файл, загружаемый в облачную систему со скрытыми правильными ответами 'positive' тестовой выборки.

**Целевое ограничение**: значение метрики **accuracy** должно быть **> 0.62**.

## **3. Решение**

---

In [1]:
# загружаем необходимые библиотеки
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# подключаем Google Drive для последующей загрузки на него
# файла с предсказаниями
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# загружаем zip-архив с обучающим и тестовым датасетами
! gdown 1XsMwM7t7FxGFN5-M74esbKOD9tF0vCk2

# разархивируем загруженный архив
! unzip -o /content/tonality_of_the_text.zip

# сохраняем датасеты в переменные
train = pd.read_csv('/content/tonality_of_the_text/tweets_lemm_train.csv')
test = pd.read_csv('/content/tonality_of_the_text/tweets_lemm_test.csv')
print('Datasets load from Google Drive storage')

Downloading...
From: https://drive.google.com/uc?id=1XsMwM7t7FxGFN5-M74esbKOD9tF0vCk2
To: /content/tonality_of_the_text.zip
100% 608k/608k [00:00<00:00, 89.6MB/s]
Archive:  /content/tonality_of_the_text.zip
  inflating: tonality_of_the_text/tweets_lemm_train.csv  
  inflating: tonality_of_the_text/tweets_lemm_test.csv  
Datasets load from Google Drive storage


Проверим, загружены ли данные для анализа, для чего выведем первые пять строк датасетов train и test.

In [3]:
train.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [4]:
test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [5]:
# выведем размеры загруженных датасетов
train.shape, test.shape

((5000, 3), (3000, 2))

Данные успешно загружены.

Чтобы алгоритм смог определить тональность текста, его необходимо обучить на корпусе - наборе текстов, в котором эмоции и ключевые слова уже размечены. Для этого необходимо перевести тексты в стандартный для Python формат - кодировку Unicode.

In [6]:
# создаем корпус слов из лемматизированных сообщений
corpus_train = train['lemm_text'].values.astype('U')

In [7]:
# создаем корпус слов из лемматизированных сообщений
corpus_test = test['lemm_text'].values.astype('U')

Тексты необходимо очистить от стоп-слов - слов, не несущих смысловой нагрузки. Для этого необходим пакет "stopwords" с аргументом "russian" (стоп-словами, характерными для русского языка).

In [8]:
# загружаем стоп-слова
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('russian')))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Оценка важности слова определяется величиной TF-IDF:

 * TF отвечает за количество упоминаний слова в отдельном тексте

 * IDF отражает частоту употребления слова во всем корпусе

Большая величина TF-IDF гооврит об уникальности слова в тексте по отношению к корпусу. Чем чаще оно встречается в конкретном корпусе и реже в остальных, тем выше значение TF-IDF.

In [9]:
# объявим счетчик величин TF-IDF
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

In [10]:
# выведем размер корпусов до применения счетчика
corpus_train.shape, corpus_test.shape

((5000,), (3000,))

In [11]:
# применим счетчик к лемматизированным текстам обучающей выборки
corpus_train = count_tf_idf.fit_transform(corpus_train)

In [12]:
# применим счетчик к лемматизированным текстам тестовой выборки
corpus_test = count_tf_idf.transform(corpus_test)

In [13]:
# выведем размер корпусов текстов после применения счетчика
corpus_train.shape, corpus_test.shape

((5000, 9737), (3000, 9737))

In [14]:
# объявляем модель
model = LogisticRegression(random_state = 1)

In [15]:
# обучаем модель
model.fit(corpus_train, train['positive'])

LogisticRegression(random_state=1)

In [16]:
# получаем предсказания модели на корпусе текстов обучающей выборки
predict_train = model.predict(corpus_train)

In [17]:
# вычислим точность предсказаний на обучающей выборке
accuracy_train = accuracy_score(train['positive'], predict_train)
print(f'accuracy на обучающей выборке:\n {accuracy_train}')

accuracy на обучающей выборке:
 0.8984


In [18]:
# получим предсказания модели на корпусе текстов тестовой выборки
predictions = pd.Series(model.predict(corpus_test), name = 'positive')

In [19]:
# выведем первые 10 строк полученных предсказаний
predictions.head()

0    1
1    0
2    0
3    0
4    1
Name: positive, dtype: int64

In [20]:
# загружаем файл с предсказаниями на Google Drive
predictions.to_csv('/content/gdrive/My Drive/predictions.csv', index = False)